In [1]:
# Dependencies
import sys
import datetime as dt
from pprint import pprint
import pandas as pd
from library.authorize.config import api_id, api_key
from library.authorize.api import UnleashedApi

In [27]:
def file_input():
    file_import = True
    while file_import:
        # User input to get excel file name
        global excel_file_name
        excel_file_name = input("Please enter in the excel file name: ")

        # Enter to exit program
        exit_strat = ["quit", "cancel", "exit"]
        if excel_file_name in exit_strat:
            sys.exit(1)

        # Excel file path
        excel_path = f"product_list_import/{excel_file_name}.xlsx"

        try:
            # Read excel file as df
            # Keep blank rows for easy copy and paste
            global product_df
            product_df = pd.read_excel(excel_path, skip_blank_lines=False)

            # Stop file_import loop
            file_import = False

        except FileNotFoundError:
            print("Sorry, there is no such file in directory. Please try again.\n")

    # Run pick_enquiry function to choose which enquiry to run
    # pick_enquiry()

    return excel_file_name, product_df

In [77]:
file_input()

Please enter in the excel file name: test


('test',   Product Code
 0     AAMDSAKT)

In [146]:
# Unleashed api base url
api_url = "https://api.unleashedsoftware.com"

# Authorize and connect to api
auth = UnleashedApi(api_url, api_id, api_key)

# Get response
bills_of_materials = auth.get_request(method="BillOfMaterials?").json()

In [147]:
def get_bom(product_df):

    # Run get_bom_response function
    print("\nExtracting bills of materials...")
    
    # Create dictionary to hold list of line items for each product in product_df
    bom_dict = {}

    # Append each product in product_df to bom_list as dictionaries
    for product in product_df["Product Code"]:
        # Add product as key with the value as a list
        bom_dict[product] = []

    # First run to grab bom line items of final product
    for product, line_list in bom_dict.items():
        for bill in bills_of_materials["Items"]:
            if bill["Product"]["ProductCode"] == product:
                for line in bill["BillOfMaterialsLines"]:
                    line_item = line["Product"]["ProductCode"]
                    if line_item != 'LBR':
                        line_list.append(line_item)
    
    pprint(bom_dict)
    
    for line_list in bom_dict.values():
        
        for i, product in enumerate(line_list):
            
            for bill in bills_of_materials["Items"]:
                
                if bill["Product"]["ProductCode"] == product:
                    
                    for line in reversed(bill["BillOfMaterialsLines"]):
                        
                        line_item = line["Product"]["ProductCode"]
                        
                        if line_item != 'LBR':# and line_item not in line_list:
                            
                            line_list.insert(i + 1, line_item)
    
    pprint(bom_dict)

    # See if we can append bom_list after each product in original excel file
    product_list = [product for product in product_df["Product Code"]]
    
    # print(product_list)
    
    # Create a final bom list
    full_bom_list = []

    # Separate final products by an em-dash
    for i, final_prod in enumerate(product_list):
        for key, value in bom_dict.items():
            if final_prod == key:
                full_bom_list.append(key)
                for line in value:
                    full_bom_list.append(line)
                full_bom_list.append("—")
                
    # print(full_bom_list)
    
    # Create bom dataframe
    bom_df = pd.DataFrame({"Product Code": full_bom_list})

    # Run get_soh function to grab descriptions and stock counts
    # get_soh(bom_df)
    return bom_df

In [148]:
get_bom(product_df)


Extracting bills of materials...
{'AAMDSAKT': ['AAMDAC30ML', 'AAMDSE30ML']}
{'AAMDSAKT': ['AAMDAC30ML',
              'BXAAMDAC30ML',
              'FBAAMDAC30ML',
              'EBAAMDAC30ML',
              'FBMDACYW30ML',
              'EBINYW30ML',
              'EBPMYW30ML',
              'FLMDACYW30ML',
              'INSCCM',
              'AAMDSE30ML',
              'BXAAMDSE30ML',
              'FBAAMDSE30ML',
              'EBAAMDSE30ML',
              'FBMDSEYW30ML',
              'EBINYW30ML',
              'EBPMYW30ML',
              'FLMDSEYW30ML',
              'INSCCM']}


,Product Code
0,AAMDSAKT
1,AAMDAC30ML
2,BXAAMDAC30ML
3,FBAAMDAC30ML
4,EBAAMDAC30ML
5,FBMDACYW30ML
6,EBINYW30ML
7,EBPMYW30ML
8,FLMDACYW30ML
9,INSCCM
